In [36]:
import numpy as np

def chol_pd(a):
    n = a.shape[0]
    root = np.zeros((n,n))

    for j in range(n):
        s = 0
        if j > 0:
            s = np.dot(root[j,:j], root[j,:j])
        root[j,j] = np.sqrt(a[j,j] - s)
        ir = 1.0 / root[j,j]
        for i in range(j+1, n):
            s = np.dot(root[i,:j], root[j,:j])
            root[i,j] = (a[i,j] - s) * ir
    return root

n = 5
sigma = np.zeros((n,n)) + 0.9
for i in range(n):
    sigma[i,i] = 1.0

root = chol_pd(sigma)

np.allclose(np.dot(root, root.T), sigma)

root2 = np.linalg.cholesky(sigma)
np.allclose(root, root2)


True

In [37]:
#using near_psd() method to fix the matrix 

import numpy as np

def near_psd(a, epsilon=0.0):
    n = a.shape[0]

    invSD = None
    out = a.copy()

    if np.count_nonzero(np.isclose(np.diag(out), 1.0)) != n:
        invSD = np.diag(1.0 / np.sqrt(np.diag(out)))
        out = np.dot(np.dot(invSD, out), invSD)

    vals, vecs = np.linalg.eigh(out)
    vals = np.maximum(vals, epsilon)
    T = 1.0 / np.sum(vecs * vecs * vals, axis=1)
    T = np.diag(np.sqrt(T))
    l = np.diag(np.sqrt(vals))
    B = np.dot(np.dot(T, vecs), l)
    out = np.dot(B, B.T)

    if invSD is not None:
        invSD = np.diag(1.0 / np.diag(invSD))
        out = np.dot(np.dot(invSD, out), invSD)

    return out


n = 500
sigma = np.full((n,n), 0.9)
for i in range(n):
    sigma[i,i] = 1.0
sigma[0,1] = 0.7357
sigma[1,0] = 0.7357

#print(sigma)
near_pairwise = near_psd(sigma)
#print(near_pairwise)
eigenvalues, _ = np.linalg.eig(near_pairwise)
print(eigenvalues)

[ 4.50043845e+02+0.00000000e+00j  2.56180531e-01+0.00000000e+00j
 -2.42987054e-14+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+9.15408171e-16j
  9.99999489e-02-9.15408171e-16j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+5.21138188e-15j
  9.99999489e-02-5.21138188e-15j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.00000000e+00j  9.99999489e-02+1.38013361e-15j
  9.99999489e-02-1.38013361e-15j  9.99999489e-02+0.00000000e+00j
  9.99999489e-02+0.000000

In [38]:
#using Higham’s method to fix the matrix 

from numpy import linalg as la
import numpy as np

def Higham(A):
 
    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

def isPD(B):
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False
    
    
#print(sigma)
fixed_matrix = Higham(sigma)
print(fixed_matrix)


[[1.03169433 0.76739433 0.89987276 ... 0.89987276 0.89987276 0.89987276]
 [0.76739433 1.03169433 0.89987276 ... 0.89987276 0.89987276 0.89987276]
 [0.89987276 0.89987276 1.00000051 ... 0.90000051 0.90000051 0.90000051]
 ...
 [0.89987276 0.89987276 0.90000051 ... 1.00000051 0.90000051 0.90000051]
 [0.89987276 0.89987276 0.90000051 ... 0.90000051 1.00000051 0.90000051]
 [0.89987276 0.89987276 0.90000051 ... 0.90000051 0.90000051 1.00000051]]


In [39]:
import numpy as np

eigenvalues, _ = np.linalg.eig(fixed_matrix)
print(eigenvalues)


[4.50099343e+02+0.00000000e+00j 2.64300000e-01+0.00000000e+00j
 2.50034368e-14+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+0.00000000e+00j
 1.00000000e-01+0.00000000e+00j 1.00000000e-01+1.87484548e-15j
 1.00000000e-01-1.87484548e-15j 1.00000000e-01+1.54957790e-14j
 1.00000000e-01-1.54957790e-14j 1.00000000e-01+4.621022

In [40]:
# print the two fixed matrix using near_psd() and Higham’s method
print(near_pairwise)
print(fixed_matrix)


[[1.         0.74381947 0.88594237 ... 0.88594237 0.88594237 0.88594237]
 [0.74381947 1.         0.88594237 ... 0.88594237 0.88594237 0.88594237]
 [0.88594237 0.88594237 1.         ... 0.90000005 0.90000005 0.90000005]
 ...
 [0.88594237 0.88594237 0.90000005 ... 1.         0.90000005 0.90000005]
 [0.88594237 0.88594237 0.90000005 ... 0.90000005 1.         0.90000005]
 [0.88594237 0.88594237 0.90000005 ... 0.90000005 0.90000005 1.        ]]
[[1.03169433 0.76739433 0.89987276 ... 0.89987276 0.89987276 0.89987276]
 [0.76739433 1.03169433 0.89987276 ... 0.89987276 0.89987276 0.89987276]
 [0.89987276 0.89987276 1.00000051 ... 0.90000051 0.90000051 0.90000051]
 ...
 [0.89987276 0.89987276 0.90000051 ... 1.00000051 0.90000051 0.90000051]
 [0.89987276 0.89987276 0.90000051 ... 0.90000051 1.00000051 0.90000051]
 [0.89987276 0.89987276 0.90000051 ... 0.90000051 0.90000051 1.00000051]]


In [41]:
#Compare the results of both using the Frobenius Norm
import numpy as np

def frobenius_norm(A):
    return np.linalg.norm(A, 'fro')

matrix1 = near_pairwise-sigma
matrix2 = fixed_matrix-sigma

print(frobenius_norm(matrix1))
print(frobenius_norm(matrix2))

0.6275226557647142
0.06364303890475485


In [46]:
#compare run time of two methods

import time

n = 5
sigma = np.zeros((n,n)) + 0.9
for i in range(n):
    sigma[i,i] = 1.0
    
    
start = time.time()
near_pairwise = near_psd(sigma)
end = time.time()
print("Time taken:", end - start, "seconds")

start = time.time()
fixed_matrix = Higham(sigma)
end = time.time()
print("Time taken:", end - start, "seconds")

Time taken: 0.0010523796081542969 seconds
Time taken: 0.0002422332763671875 seconds


In [48]:
#How does the run time of each function compare as N increases?
n = 10
sigma = np.zeros((n,n)) + 0.9
for i in range(n):
    sigma[i,i] = 1.0

start = time.time()
near_pairwise = near_psd(sigma)
end = time.time()
print("Time taken:", end - start, "seconds")

start = time.time()
fixed_matrix = Higham(sigma)
end = time.time()
print("Time taken:", end - start, "seconds")

Time taken: 0.001531362533569336 seconds
Time taken: 0.0004818439483642578 seconds
